In [364]:
import numpy as np 
import torch

batch_size = 5


At first we have : 
- camera's parameters -> A (focal, center) 
- rotation matrix  -> R 
- position matrix -> C 
- 3D points position ->  P1 P2 P3 (and P4 to determinate the best solution after P3P)

In [365]:
# This script defines the camera parameters, rotation matrix, and translation matrix.
def camera() : 
  # Definition of the camera parameters
  # focal length
  fx = 800
  fy = 800
  # center
  cx = 320 
  cy = 240

  A = torch.tensor([[fx, 0, cx], [0, fy, cy], [0, 0, 1]], dtype=torch.float64) # intraseca matrix of the camera (3*3)
  
  A_batch = A.repeat(batch_size,1,1)
  print("A_batch = \n", A_batch)
  print(A_batch.shape)  # (batch_size, 3, 3)
  return A_batch

A = camera() 



def rotation_matrix() : 
  # Definition of the rotation matrix of the camera 
  R = torch.tensor([[1, 0, 0],[0, -1, 0], [0, 0, -1]], dtype=torch.float64)  # (3*3)
  
  R_batch = R.repeat(batch_size,1,1)  # Repeat the rotation matrix for each batch
  print("R_batch = \n", R_batch)  
  print(R_batch.shape)  # (batch_size, 3, 3)
  return R_batch

def camera_position() : 
  # Definition of the translation matrix of the camera (the position)
  C = torch.tensor([[0,0,6]], dtype=torch.float64)    # T = [tx,ty,tz]  (1*3)
  

  C_batch = C.repeat(batch_size, 1)  # Repeat the translation vector for each batch
  print("C_batch = \n", C_batch)  
  print(C_batch.shape)  # (batch_size, 3)
  return C_batch

R = rotation_matrix()
C = camera_position()
print("batch_size = ", batch_size)


A_batch = 
 tensor([[[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]],

        [[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]],

        [[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]],

        [[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]],

        [[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]]], dtype=torch.float64)
torch.Size([5, 3, 3])
R_batch = 
 tensor([[[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]],

        [[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]],

        [[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]],

        [[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]],

        [[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]]], dtype=torch.float64)
torch.Size([5, 3, 3])
C_batch = 
 tensor([[0., 0., 6.],
        [0.

In [366]:
#torch.manual_seed(1234)  

# Definition of 3D points in the world coordinate system
def point3Daleatoire(x) :
  # Generation of one random points in 3D space 
  return torch.empty((1, 3), dtype=torch.float64).uniform_(-x, x)


def pts_3D_4pts():
  # Generate randomly 4 3D points
  # Output : tensor which concatenate the 4 points = [ P1, P2, P3, P4 ] 

  P1 = point3Daleatoire(2)     # (1*3) -> pour P3P
  P2 = point3Daleatoire(2)
  P3 = point3Daleatoire(2)
  P4 = point3Daleatoire(2)
  
  points3D = torch.cat((P1,P2,P3,P4),dim=0);     # (LIGNES 4* COLONNES 3) - xyz
    
  return points3D

def pts_3D_4pts_batch():
  # Generate randomly 4 3D points for each batch
  # Output : array which concatenate the 4 points = [ P1, P2, P3, P4 ] for each batch

  # Generate a batch of random points in 3D space
  # Each point is generated independently for each batch

  points3D_batch = torch.stack([pts_3D_4pts() for i in range(batch_size)])  # (batch_size, 4, 3)
  
  return points3D_batch

points3D_batch = pts_3D_4pts_batch()  # Generate the batch of 3D points
'''
P1 = torch.tensor([0.7161, 0.5431, 1.7807], dtype=torch.float64)    # (3,)
P2 = torch.tensor([-1.1643, 0.8371, -1.0551], dtype=torch.float64)
P3 = torch.tensor([-1.5224, 0.4292, -0.1994], dtype=torch.float64)
P4 = torch.tensor([-1.5224, 0.4292, -0.1994], dtype=torch.float64) 
'''
P1 = points3D_batch[:, 0, :]  # Extract P1 for each batch
P2 = points3D_batch[:, 1, :]  # Extract P2 for each batch
P3 = points3D_batch[:, 2, :]  # Extract P3 for each batch
P4 = points3D_batch[:, 3, :]  # Extract P4 for each batch

print("P1 = \n", P1)
print("P2 = \n", P2)
print("P3 = \n", P3)
print("P4 = \n", P4)
print(P1.shape)  # (batch_size, 3)


P1 = 
 tensor([[-0.5443,  0.4731, -1.0539],
        [-0.4108,  1.5045, -1.5280],
        [ 1.7787,  0.6991,  1.4326],
        [-0.8507, -1.1777,  0.8767],
        [-0.6779, -1.9685,  0.6495]], dtype=torch.float64)
P2 = 
 tensor([[ 0.2480,  0.8144,  1.2974],
        [ 0.8775,  1.3337, -0.0562],
        [ 1.5681, -0.8198,  0.1086],
        [ 0.6003,  0.3116, -1.5036],
        [ 1.5275,  0.4889,  1.6081]], dtype=torch.float64)
P3 = 
 tensor([[-1.8059, -1.2870,  0.2262],
        [-0.4282,  0.0530,  1.2429],
        [ 0.7657, -1.1758, -1.0990],
        [-0.1201,  1.0708, -1.4293],
        [ 0.9256, -0.6729,  0.6917]], dtype=torch.float64)
P4 = 
 tensor([[-0.7159,  1.8139,  1.9869],
        [ 1.4040,  0.7298,  1.5383],
        [ 1.8534,  0.7321, -1.5068],
        [ 1.1135,  0.6070,  1.2114],
        [ 0.2160, -1.2766,  0.5363]], dtype=torch.float64)
torch.Size([5, 3])


We create the 3 direction features vectors f1, f2, f3

In [367]:
def features_vectors(points3D,C, R,batch_size) :
    '''
    This function computes the features vectors for P3P algorithm.
    args:
    points3D : array with the 4 3D points = [ P1, P2, P3, P4 ] (4*3) 
    but we only use the first three points for P3P
    C: camera position matrix : (3*1)
    returns:
    featuresVect : array with the features vectors (9*1)
    '''
    P1 = torch.reshape(points3D[0], (batch_size,3,1))  # (batch_size, 3, 1)
    P2 = torch.reshape(points3D[1], (batch_size,3,1))
    P3 = torch.reshape(points3D[2], (batch_size,3,1))

    C = torch.reshape(C, (batch_size,3,1))   # (batch_size, 3, 1)
    
    v1 = torch.matmul(R,(P1 - C))  # (batch_size, 3, 1)
    v2 = torch.matmul(R,(P2 - C))  # (batch_size, 3, 1)
    v3 = torch.matmul(R,(P3 - C))  # (batch_size, 3, 1)

    f1 = v1 / torch.norm(v1,dim=1, keepdim=True)  # (batch_size, 3, 1)
    f2 = v2 / torch.norm(v2,dim=1, keepdim=True)  # (batch_size, 3, 1)
    f3 = v3 / torch.norm(v3,dim=1, keepdim=True)  # (batch_size, 3, 1)


    f1 = torch.reshape(f1, (batch_size,1,3))  # (batch_size,1,3)
    f2 = torch.reshape(f2, (batch_size,1,3))
    f3 = torch.reshape(f3, (batch_size,1,3))

    featuresVect = torch.cat((f1,f2,f3),dim=1) # (batch_size, 3, 3)
    print("features vectors = \n",featuresVect)
    print(featuresVect.shape)  

    return featuresVect # Return the features vectors need in P3P


points3D = [P1, P2, P3]  # We define the points3D with the first three points
print("points3D = \n", points3D)  # Print the points3D to check the values  / List len = 3 

featuresVect = features_vectors(points3D, C, R,batch_size)  

points3D = 
 [tensor([[-0.5443,  0.4731, -1.0539],
        [-0.4108,  1.5045, -1.5280],
        [ 1.7787,  0.6991,  1.4326],
        [-0.8507, -1.1777,  0.8767],
        [-0.6779, -1.9685,  0.6495]], dtype=torch.float64), tensor([[ 0.2480,  0.8144,  1.2974],
        [ 0.8775,  1.3337, -0.0562],
        [ 1.5681, -0.8198,  0.1086],
        [ 0.6003,  0.3116, -1.5036],
        [ 1.5275,  0.4889,  1.6081]], dtype=torch.float64), tensor([[-1.8059, -1.2870,  0.2262],
        [-0.4282,  0.0530,  1.2429],
        [ 0.7657, -1.1758, -1.0990],
        [-0.1201,  1.0708, -1.4293],
        [ 0.9256, -0.6729,  0.6917]], dtype=torch.float64)]
features vectors = 
 tensor([[[-0.0768, -0.0667,  0.9948],
         [ 0.0519, -0.1704,  0.9840],
         [-0.2920,  0.2081,  0.9335]],

        [[-0.0534, -0.1957,  0.9792],
         [ 0.1401, -0.2129,  0.9670],
         [-0.0896, -0.0111,  0.9959]],

        [[ 0.3593, -0.1412,  0.9225],
         [ 0.2549,  0.1333,  0.9577],
         [ 0.1058,  0.1625,  0.98

Lastly we need the functions to resolve the polynomial roots. - for test go to test resolution polynome 

In [368]:
from complex_batch_utils import *
import complex_utils as cpu
from torch import vmap

def polynomial_root_calculation_3rd_degree(a, b, c, d):
    # This function calculates the roots of a cubic polynomial of the form:
    # a*x^3 + b*x^2 + c*x + d = 0
        # a (batch_size, 1)
        # b (batch_size, 1)
        # c (batch_size, 1)
        # d (batch_size, 1)

    # output: roots of the polynomial in the form of a tensor of shape (batch_size,3, 2)
    # where each root is represented as a complex number (real, imaginary)
    # each row is the i_th root of the polynomial
    
    batch_size = a.shape[0]  # Get the batch size from the shape of a
    

    # Discriminant terms
    p = (3 * a * c - b**2) / (3 * a**2)     # (batch_size, 1) because element-wise opeations
    q = (2 * b**3 - 9 * a * b * c + 27 * a**2 * d) / (27 * a**3)    # (batch_size, 1)
    delta = -4 * p**3 - 27 * q**2   # (batch_size, 1)

    roots = torch.empty((batch_size,3, 2))  # Initialize roots tensor to store the roots

    j_ = torch.tensor([-0.5, torch.sqrt(torch.tensor(3))/2])  # cube root of unity
    
    for k in range (3):
        delta_sur_27 = -delta / 27   #(batch_size, 1) 

        sqrt_term = sqrt_batch(delta_sur_27)  

        # faire une seule fois les calculs de j^k et j^-k
        j_exp_k = cpu.complex_number_power_k(j_, k)  # Compute j^k for each batch
        j_exp_moins_k = cpu.complex_number_power_k(j_, -k)  # Compute j^-k for each batch

        j_exp_k_batch = j_exp_k.repeat(batch_size, 1)
        j_exp_moins_k_batch = j_exp_moins_k.repeat(batch_size, 1)

        u_k = product_of_2_complex_numbers_batch(j_exp_k_batch, sqrt_3_batch(torch.stack([0.5*(-q.squeeze()+sqrt_term[:,0]),sqrt_term[:,1]],dim=-1)) )
         # (batch_size, 2) 
        v_k = product_of_2_complex_numbers_batch(j_exp_moins_k_batch, sqrt_3_batch(torch.stack([0.5*(-q.squeeze()-sqrt_term[:,0]),-0.5*sqrt_term[:,1]],dim=-1)))
          # (batch_size, 2) 

        root = addition_batch(addition_batch(u_k, v_k), torch.stack([-b[:,0]/(3*a[:,0]),0.0*b[:,0]],dim=-1) ) 
         # (batch_size, 2)

        roots[:,k,:] = root  # Store the root in the roots tensor

    return roots


In [369]:

def polynomial_root_calculation_4th_degree_ferrari(a0, a1, a2, a3, a4): # Ferrari's Method
       
    # Solving a polynomial of 4th degree
    # a0, a1, a2, a3, a4 (batch_size,1)

    # Output : roots of the polynomial a4*x^4 + a3*x^3 + a2*x^2 + a1*x + a0   (4,batch_size 2) !! ATTENTION !!! 

    batch_size = a0.shape[0]  # Get the batch size from the shape of a0

    # Reduce the quartic equation to the form : x^4 + a*x^3 + b*x^2 + c*x + d = 0
    a = a3/a4           # (batch_size, 1)
    b = a2/a4
    c = a1/a4
    d = a0/a4

    # Computation of the coefficients of the Ferrari's Method
    S = a/4     # (batch_size, 1)
    b0 = d - c*S + b* S**2 - 3* S**4    # (batch_size, 1)
    b1 = c - 2*b*S + 8*S**3 # (batch_size, 1)
    b2 = b - 6 * S**2  # (batch_size, 1)


    # Solve the cubic equation m^3 + b2*m^2 + (b2^2/4  - b0)*m - b1^2/8 = 0
    x_cube = polynomial_root_calculation_3rd_degree(torch.tensor(1).repeat(b2.shape),b2,(b2**2)/4-b0,(-b1**2)/8)
  
    
    x_cube_real = x_cube[:,:,0] #  (batch_size, 3)
    x_cube_imag = x_cube[:,:,1]  #    (batch_size, 3)

    is_real = torch.isclose(x_cube_imag,torch.tensor(0.0))
    is_positive = x_cube[:,:,0] > 0
    condition = is_real & is_positive  # Condition to check if the root is real and positive   (batch_size, 3)
   
    
    real_filtered = x_cube_real.clone()
    real_filtered[~condition] = float('inf')  # if root real and positive, keep it, else set to infinity   (batch_size, 3)


    alpha_0_real, _ = real_filtered.min(dim=1) # Get the minimum real part of the roots (if doesn't exist, returns inf)

    alpha_0 = torch.stack([alpha_0_real, torch.zeros(batch_size)], dim=-1)  # (batch_size,2)
    

    # do the calculation for alpha_0_nul and not alpha_0_nul and then affects the good value 

    # if alpha_0_nul == False
    alpha0_div_2 = 0.5*alpha_0              # beacause alpha_0 is real  # (batch_size, 2)    
    sqrt_alpha = sqrt_batch(alpha0_div_2[:,0].unsqueeze(-1))    # input : (batch_size, 1) // output : (batch_size, 2)
    term = addition_complex_real_batch(- alpha0_div_2 ,-b2 / 2)      # (batch_size, 2)
    denom = 2 * torch.sqrt(2 * alpha_0)      # beacause alpha_0 is real  # (batch_size, 2)
    num = torch.stack([b1, torch.zeros(batch_size,1)], dim=-1).squeeze(1)  # (batch_size, 2)
   

    frac = division_2_complex_numbers(num,denom)    # (batch_size, 2)
    
    x1_false = addition_complex_real_batch(sqrt_alpha ,- S) + sqrt_complex_batch(addition_batch(term,-frac))    # (batch_size, 2)
    x2_false = addition_complex_real_batch(sqrt_alpha, - S) - sqrt_complex_batch(addition_batch(term,-frac))    # (batch_size, 2)
    x3_false = addition_complex_real_batch(-sqrt_alpha, - S) + sqrt_complex_batch(addition_batch(term,frac))    # (batch_size, 2)
    x4_false = addition_complex_real_batch(-sqrt_alpha,- S) - sqrt_complex_batch(addition_batch(term,frac))     # (batch_size, 2)
    
    # if alpha_0_nul == True
    sqrt_inner1 = sqrt_batch((b2**2) / 4 - b0)        # complex 

    x1_true = addition_complex_real_batch(sqrt_complex_batch(addition_complex_real_batch(sqrt_inner1,-b2 / 2)),-S)
    x2_true = addition_complex_real_batch(- sqrt_complex_batch(addition_complex_real_batch(sqrt_inner1,-b2 / 2)),-S)
    x3_true = addition_complex_real_batch(sqrt_complex_batch(addition_complex_real_batch(- sqrt_inner1,-b2 / 2 )),-S)
    x4_true = addition_complex_real_batch(- sqrt_complex_batch(addition_complex_real_batch(- sqrt_inner1,-b2 / 2 )),-S)
    
    result = torch.where(alpha_0==float('inf'),
                         torch.stack([x1_true, x2_true, x3_true, x4_true]),torch.stack([x1_false, x2_false, x3_false, x4_false]))
    
    return result   # (4,batch_size, 2)

a0  = torch.rand(batch_size, 1, dtype=torch.float64)
a1  = torch.rand(batch_size, 1, dtype=torch.float64)  
a2  = torch.rand(batch_size, 1, dtype=torch.float64)  
a3  = torch.rand(batch_size, 1, dtype=torch.float64) 
a4  = torch.rand(batch_size, 1, dtype=torch.float64)  


We have all the variables needed for the p3p so we start

1. Storage of points : already done 

In [370]:
print("P1 = \n", P1)
print("P2 = \n", P2)
print("P3 = \n", P3)

P1 = 
 tensor([[-0.5443,  0.4731, -1.0539],
        [-0.4108,  1.5045, -1.5280],
        [ 1.7787,  0.6991,  1.4326],
        [-0.8507, -1.1777,  0.8767],
        [-0.6779, -1.9685,  0.6495]], dtype=torch.float64)
P2 = 
 tensor([[ 0.2480,  0.8144,  1.2974],
        [ 0.8775,  1.3337, -0.0562],
        [ 1.5681, -0.8198,  0.1086],
        [ 0.6003,  0.3116, -1.5036],
        [ 1.5275,  0.4889,  1.6081]], dtype=torch.float64)
P3 = 
 tensor([[-1.8059, -1.2870,  0.2262],
        [-0.4282,  0.0530,  1.2429],
        [ 0.7657, -1.1758, -1.0990],
        [-0.1201,  1.0708, -1.4293],
        [ 0.9256, -0.6729,  0.6917]], dtype=torch.float64)


2. Storage of the features vectors : done

In [371]:
# we got featuresVect and we access the 3 values 
f1 = featuresVect[:,0,:]  # Access the first feature vector for each batch
f2 = featuresVect[:,1,:]
f3 = featuresVect[:,2,:]

print("f1 = ", f1)
print(f1.shape)  # (batsh_size,3)
print("f2 = ", f2)
print(f2.shape)  # (batsh_size,3)
print("f3 = ", f3)
print(f3.shape)  # (batsh_size,3)

f1 =  tensor([[-0.0768, -0.0667,  0.9948],
        [-0.0534, -0.1957,  0.9792],
        [ 0.3593, -0.1412,  0.9225],
        [-0.1597,  0.2212,  0.9621],
        [-0.1181,  0.3429,  0.9319]], dtype=torch.float64)
torch.Size([5, 3])
f2 =  tensor([[ 0.0519, -0.1704,  0.9840],
        [ 0.1401, -0.2129,  0.9670],
        [ 0.2549,  0.1333,  0.9577],
        [ 0.0797, -0.0414,  0.9960],
        [ 0.3267, -0.1046,  0.9393]], dtype=torch.float64)
torch.Size([5, 3])
f3 =  tensor([[-0.2920,  0.2081,  0.9335],
        [-0.0896, -0.0111,  0.9959],
        [ 0.1058,  0.1625,  0.9810],
        [-0.0160, -0.1426,  0.9896],
        [ 0.1705,  0.1239,  0.9775]], dtype=torch.float64)
torch.Size([5, 3])


3. Création of a solution variable : maximum 4 solutions  

    Matrix (4,3,4)  
    Each layer is a solution, for each leayer : first column stres the camera position matrix C (3,1) and the remaining 3 columns store the rotation matrix R (3,3)

In [372]:
solutions = torch.empty((batch_size,4,3,4), dtype=torch.float64)

print("solutions = \n", solutions)
print(solutions.shape)  # (batch_size,4,3,4)

solutions = 
 tensor([[[[1.5169e-311, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324]],

         [[4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324]],

         [[4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324]],

         [[4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324],
          [4.9407e-324, 4.9407e-324, 4.9407e-324, 4.9407e-324]]],


        [[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.

4. Verification that the 3 points given are not collinear 

In [373]:
# Test of non-collinearity
v1 = P2 - P1 # (batch_size, 3)
 
v2 = P3 - P1 # (batch_size, 3)

norms = torch.norm(torch.cross(v1,v2, dim=1),dim = 1 )

all_dif_zero = torch.all(norms != 0)  

if not all_dif_zero:
    print('\nProblem: the points must not be collinear')
else:
    print('\nThe points are not collinear, we can continue')


The points are not collinear, we can continue


5. Creation of an orthonormal frame from f1, f2, f3 (the features vectors)

In [374]:
# Calculation of vectors of the base τ = (C,tx,ty,tz)
tx = f1     # (batch_size,3)
print("tx = ", tx)
print(tx.shape)  # (batch_size,3)

tz = torch.cross(f1,f2,dim=1)/ torch.norm(torch.cross(f1,f2,dim=1),dim=1, keepdim=True) 
# (batch_size,3)
print("tz = ", tz)
print(tz.shape)  # (batch_size,3)

ty = torch.cross(tz,tx,dim=1) # (batch_size,3)
print("ty = ", ty)
print(ty.shape)  # (batch_size,3)

tx =  tensor([[-0.0768, -0.0667,  0.9948],
        [-0.0534, -0.1957,  0.9792],
        [ 0.3593, -0.1412,  0.9225],
        [-0.1597,  0.2212,  0.9621],
        [-0.1181,  0.3429,  0.9319]], dtype=torch.float64)
torch.Size([5, 3])
tz =  tensor([[ 0.6294,  0.7706,  0.1002],
        [ 0.0995,  0.9747,  0.2002],
        [-0.8827, -0.3723,  0.2868],
        [ 0.7405,  0.6713, -0.0314],
        [ 0.7007,  0.6938, -0.1665]], dtype=torch.float64)
torch.Size([5, 3])
ty =  tensor([[ 0.7732, -0.6339,  0.0172],
        [ 0.9936, -0.1081,  0.0326],
        [-0.3030,  0.9173,  0.2584],
        [ 0.6528, -0.7074,  0.2710],
        [ 0.7036, -0.6333,  0.3222]], dtype=torch.float64)
torch.Size([5, 3])


5. (bis) Creation of a transformation matrix T and expression of the f3 vector in this frame

In [375]:
tx = torch.reshape(tx,(batch_size,1,3))   # (batch_size,1,3)
print("tx = \n", tx)

ty = torch.reshape(ty,(batch_size,1,3))  # (batch_size,1,3)
print("ty = \n", ty)

tz = torch.reshape(tz,(batch_size,1,3))  # (batch_size,1,3)
print("tz = \n", tz)

# Computation of the matrix T and the feature vector f3
T = torch.cat((tx,ty,tz),dim = 1) # (3*3)
print("T = \n", T)
print(T.shape)  # (batch_size,3,3)


f3_T = torch.matmul(T,f3.unsqueeze(-1)) # (
print("f3_T = \n", f3_T)
print(f3_T.shape)  # (batch_size,3,1)


tx = 
 tensor([[[-0.0768, -0.0667,  0.9948]],

        [[-0.0534, -0.1957,  0.9792]],

        [[ 0.3593, -0.1412,  0.9225]],

        [[-0.1597,  0.2212,  0.9621]],

        [[-0.1181,  0.3429,  0.9319]]], dtype=torch.float64)
ty = 
 tensor([[[ 0.7732, -0.6339,  0.0172]],

        [[ 0.9936, -0.1081,  0.0326]],

        [[-0.3030,  0.9173,  0.2584]],

        [[ 0.6528, -0.7074,  0.2710]],

        [[ 0.7036, -0.6333,  0.3222]]], dtype=torch.float64)
tz = 
 tensor([[[ 0.6294,  0.7706,  0.1002]],

        [[ 0.0995,  0.9747,  0.2002]],

        [[-0.8827, -0.3723,  0.2868]],

        [[ 0.7405,  0.6713, -0.0314]],

        [[ 0.7007,  0.6938, -0.1665]]], dtype=torch.float64)
T = 
 tensor([[[-0.0768, -0.0667,  0.9948],
         [ 0.7732, -0.6339,  0.0172],
         [ 0.6294,  0.7706,  0.1002]],

        [[-0.0534, -0.1957,  0.9792],
         [ 0.9936, -0.1081,  0.0326],
         [ 0.0995,  0.9747,  0.2002]],

        [[ 0.3593, -0.1412,  0.9225],
         [-0.3030,  0.9173,  0.2584],
  

The sing of the z-coordinate in f3_T give us the sign of teta, that we will need after 

In [376]:
print(f3_T[:,2])
f3_T_positif = f3_T[:,2] > 0        # (batch_size,1)

print("f3_T_positif = \n", f3_T_positif)
print(f3_T_positif.shape)  

tensor([[ 0.0701],
        [ 0.1797],
        [ 0.1274],
        [-0.1386],
        [ 0.0427]], dtype=torch.float64)
f3_T_positif = 
 tensor([[ True],
        [ True],
        [ True],
        [False],
        [ True]])
torch.Size([5, 1])


6. Change of frame is performed on the 3D points side, and the transformation matrix N is defined

In [377]:
# Calculation of vectors of the base η = (P1,nx,ny,nz)
nx = (P2 - P1)/torch.norm(P2 - P1,dim=1,keepdim=True)      #(batch_size,3)
nz = torch.cross(nx,P3-P1,dim=1)/torch.norm(torch.cross(nx,P3-P1,dim=1), dim=1, keepdim=True)  # (batch_size,3)
ny = torch.cross(nz,nx,dim=1) # (batch_size,3)



# Reshape the vectors to (1,3) for concatenation
nx = torch.reshape(nx,(batch_size,1,3))  # (batch_size,1,3)
ny = torch.reshape(ny,(batch_size,1,3))
nz = torch.reshape(nz,(batch_size,1,3))

print("nx = \n", nx)
print(nx.shape)  # (1*3)
print("ny = \n", ny)
print("nz = \n", nz)

# Computation of the matrix N and the world point P3
N = torch.cat((nx,ny,nz),dim = 1) #  T's equivalent in the world coordinate system
print("N = \n", N)
print(N.shape)  # (batch_size,3,3)

print("P3.shape = ", P3.shape)  # (batch_size,3)

P3_n = torch.matmul(N,(P3-P1).unsqueeze(-1)) 


print("P3_n = \n", P3_n)
print(P3_n.shape)  # (5,3,1)


nx = 
 tensor([[[ 0.3163,  0.1363,  0.9388]],

        [[ 0.6561, -0.0870,  0.7496]],

        [[-0.1040, -0.7497, -0.6535]],

        [[ 0.4591,  0.4712, -0.7531]],

        [[ 0.6414,  0.7147,  0.2788]]], dtype=torch.float64)
torch.Size([5, 1, 3])
ny = 
 tensor([[[-0.5872, -0.7491,  0.3066]],

        [[-0.6523, -0.5649,  0.5054]],

        [[-0.7091,  0.5166, -0.4799]],

        [[-0.6742,  0.7369,  0.0500]],

        [[ 0.5514, -0.1769, -0.8153]]], dtype=torch.float64)
nz = 
 tensor([[[ 0.7451, -0.6482, -0.1570]],

        [[ 0.3795, -0.8205, -0.4274]],

        [[ 0.6974,  0.4135, -0.5853]],

        [[ 0.5785,  0.4848,  0.6560]],

        [[-0.5334,  0.6767, -0.5076]]], dtype=torch.float64)
N = 
 tensor([[[ 0.3163,  0.1363,  0.9388],
         [-0.5872, -0.7491,  0.3066],
         [ 0.7451, -0.6482, -0.1570]],

        [[ 0.6561, -0.0870,  0.7496],
         [-0.6523, -0.5649,  0.5054],
         [ 0.3795, -0.8205, -0.4274]],

        [[-0.1040, -0.7497, -0.6535],
         [-0.7091,

7. Definition of the variables for the following steps 

In [378]:
# Computation of phi1 et phi2 with 0=x, 1=y, 2=z
phi1 = f3_T[:,0]/f3_T[:,2]  # (batch_size,1)
phi2 = f3_T[:,1]/f3_T[:,2]  # (batch_size,1)
print("phi1 = ", phi1)
print("phi2 = ", phi2)

# Extraction of p1 and p2 from P3_eta
p1 = P3_n[:,0] #x  # (batch_size,3)
p2 = P3_n[:,1] #y  # (batch_size,3)
print("p1 = ", p1)
print("p2 = ", p2)

# Computation of d12
d12 = torch.norm(P2-P1,dim=1, keepdim=True)  # (batch_size,1)
print("d12 = ", d12)

# Computation of b = cot(beta)
cosBeta =( torch.sum(f1*f2,dim=1)/(torch.norm(f1,dim=1)*torch.norm(f2,dim=1)) ).unsqueeze(-1)  # tensor.dot(a,b) <=> tensor.sum(a*b)   # (batch_size,1)
print("cosBeta = ", cosBeta)  

b = torch.sqrt(1/(1-cosBeta**2)-1)

b = torch.where(cosBeta < 0, -b, b)  # If cosBeta < 0, then b = -b    # (batch_size,1)
print("b = ", b)


phi1 =  tensor([[13.3634],
        [ 5.4668],
        [ 7.2209],
        [-6.6580],
        [21.8711]], dtype=torch.float64)
phi2 =  tensor([[-4.8716],
        [-0.3083],
        [ 2.9076],
        [-2.5868],
        [ 8.3513]], dtype=torch.float64)
p1 =  tensor([[0.5628],
        [2.1919],
        [3.1654],
        [3.1316],
        [1.9664]], dtype=torch.float64)
p2 =  tensor([[2.4518],
        [2.2316],
        [0.9646],
        [1.0489],
        [0.6207]], dtype=torch.float64)
d12 =  tensor([[2.5046],
        [1.9634],
        [2.0259],
        [3.1606],
        [3.4383]], dtype=torch.float64)
cosBeta =  tensor([[0.9863],
        [0.9810],
        [0.9563],
        [0.9363],
        [0.8010]], dtype=torch.float64)
b =  tensor([[5.9765],
        [5.0630],
        [3.2695],
        [2.6661],
        [1.3378]], dtype=torch.float64)


8. Calculation of the coefficients of the polynomial 

In [379]:
a4 = - phi2**2 * p2**4 - phi1**2 * p2**4 - p2**4
a3 = 2 * p2**3 * d12 * b + 2 * phi2**2 * p2**3 * d12 * b - 2 * phi1 * phi2 * p2**3 * d12
a2 = - phi2**2 * p1**2 * p2**2 - phi2**2 * p2**2 * d12**2 * b**2 - phi2**2 * p2**2 * d12**2 + phi2**2 * p2**4 + phi1**2 * p2 **4 + 2 * p1 * p2**2 * d12 + 2 * phi1 * phi2 * p1 * p2**2 * d12 * b - phi1**2 * p1**2 * p2**2 + 2 * phi2**2 * p1 * p2**2 * d12 - p2**2 * d12**2 * b**2 - 2 * p1**2 * p2**2
a1 = 2 * p1**2 * p2 * d12 * b + 2 * phi1 * phi2 * p2**3 * d12 - 2 * phi2**2 * p2**3 * d12 * b - 2 * p1 * p2 * d12**2 * b
a0 = - 2 * phi1 * phi2 * p1 * p2**2 * d12 * b + phi2**2 * p2**2 * d12**2 + 2 * p1**3 * d12 - p1**2 * d12**2 + phi2**2 * p1**2 * p2**2 - p1**4 - 2 * phi2**2 * p1 * p2**2 * d12 + phi1**2 * p1**2 * p2**2 + phi2**2 * p2**2 * d12**2 * b**2

print("a4 = ", a4)
print(a4.shape)  # (batch_size,1)
print("a3 = ", a3)
print("a3.shape = ", a3.shape)  # (batch_size,1)
print("a2 = ", a2)
print(a2.shape)  # (batch_size,1)
print("a1 = ", a1)
print(a1.shape)  # (batch_size,1)
print("a0 = ", a0)
print(a0.shape)  # (batch_size,1)


a4 =  tensor([[-7346.9046],
        [ -768.4041],
        [  -53.3280],
        [  -62.9590],
        [  -81.4828]], dtype=torch.float64)
torch.Size([5, 1])
a3 =  tensor([[15719.2372],
        [  315.5269],
        [   36.0576],
        [   23.9522],
        [ -144.7012]], dtype=torch.float64)
a3.shape =  torch.Size([5, 1])
a2 =  tensor([[-3.3461e+04],
        [-8.8123e+02],
        [-1.7334e+01],
        [-1.7165e+01],
        [ 9.0801e+00]], dtype=torch.float64)
torch.Size([5, 1])
a1 =  tensor([[-1.5358e+04],
        [-7.2337e+01],
        [ 2.1925e+01],
        [-6.1113e+00],
        [ 1.3038e+02]], dtype=torch.float64)
torch.Size([5, 1])
a0 =  tensor([[3.9436e+04],
        [1.1274e+03],
        [9.2508e+00],
        [9.8630e-01],
        [5.7957e+01]], dtype=torch.float64)
torch.Size([5, 1])


9. Recovery of the polynomial roots cos (teta)

In [380]:
from torch import vmap

# Computation of the roots
roots = polynomial_root_calculation_4th_degree_ferrari(a0,a1,a2,a3,a4) # (batch_size,4)

print("roots = \n", roots)  # list of tensor (for complex numbers)


roots = 
 tensor([[[ 1.0708e+00,  2.0935e+00],
         [ 1.6335e-01,  1.3665e+00],
         [ 9.6911e-01,  0.0000e+00],
         [ 2.8175e-01,  5.7817e-01],
         [ 9.4919e-01,  0.0000e+00]],

        [[ 1.0708e+00, -2.0935e+00],
         [ 1.6335e-01, -1.3665e+00],
         [-2.9382e-01,  0.0000e+00],
         [ 2.8175e-01, -5.7817e-01],
         [-9.2768e-01,  0.0000e+00]],

        [[ 9.8423e-01,  0.0000e+00],
         [ 9.2310e-01,  0.0000e+00],
         [ 4.3197e-04,  7.8052e-01],
         [ 1.2352e-01,  0.0000e+00],
         [-8.9868e-01,  1.2067e-02]],

        [[-9.8630e-01, -0.0000e+00],
         [-8.3919e-01, -0.0000e+00],
         [ 4.3204e-04, -7.8052e-01],
         [-3.0659e-01, -0.0000e+00],
         [-8.9868e-01, -1.2067e-02]]], dtype=torch.float64)


10. For each solution : computation of the camera position and rotation matrix

In [381]:
# For each solution of the polynomial
for i in range(4):
  # Computation of trigonometrics forms
  cos_teta = (roots[i,:,0]).unsqueeze(-1) # real part of the root (batch_size,1)

  print("cos_teta = \n", cos_teta)   # (batch_size,1)
  sin_teta = torch.where(f3_T_positif,-torch.sqrt(1-cos_teta**2),torch.sqrt(1-cos_teta**2))
  print("sin_teta = \n", sin_teta) # (batch_size,1)

  cot_alpha = ((phi1/phi2)*p1 + cos_teta*p2 -d12*b )/ ((phi1/phi2)*cos_teta* p2 - p1 + d12)

  sin_alpha = torch.sqrt(1/(cot_alpha**2+1))
  cos_alpha = torch.sqrt(1-sin_alpha**2)
  cos_alpha = torch.where(cot_alpha < 0, -cos_alpha, cos_alpha)  

  # Computation of the intermediate rotation's matrixs
  C_estimate = torch.stack([d12*cos_alpha*(sin_alpha*b + cos_alpha), d12*sin_alpha*cos_teta*(sin_alpha*b+cos_alpha), d12*sin_alpha*sin_teta*(sin_alpha*b+cos_alpha)],dim=1)  # (batch_size,3,1)
  # (batch_size,3,1)

  Q_row1 = torch.stack([-cos_alpha, -sin_alpha*cos_teta, -sin_alpha*sin_teta], dim=-1)  
  Q_row2 = torch.stack([sin_alpha, -cos_alpha*cos_teta, -cos_alpha*sin_teta],dim=-1)
  Q_row3 = torch.stack([0*sin_teta, -sin_teta, cos_teta], dim=-1)
  Q = torch.stack([Q_row1,Q_row2 ,Q_row3],dim=1).squeeze(2)  # (batch_size,3*3)
  


  # Computation of the absolute camera center
  C_estimate = P1.unsqueeze(-1) + torch.matmul(torch.transpose(N, 1,2), C_estimate) # (batch_size,3,1)
 
  

  # Computation of the orientation matrix
  R_estimate = torch.matmul(torch.matmul(torch.transpose(N,1,2),torch.transpose(Q, 1,2)),T)   # (batch_size,3,3)

  # Adding C and R to the solutions
  solutions[:,i,:,:1]= C_estimate
  solutions[:,i,:,1:] = R_estimate

print("solutions = \n", solutions)


cos_teta = 
 tensor([[1.0708],
        [0.1634],
        [0.9691],
        [0.2818],
        [0.9492]], dtype=torch.float64)
sin_teta = 
 tensor([[    nan],
        [-0.9866],
        [-0.2466],
        [ 0.9595],
        [-0.3147]], dtype=torch.float64)
cos_teta = 
 tensor([[ 1.0708],
        [ 0.1634],
        [-0.2938],
        [ 0.2818],
        [-0.9277]], dtype=torch.float64)
sin_teta = 
 tensor([[    nan],
        [-0.9866],
        [-0.9559],
        [ 0.9595],
        [-0.3734]], dtype=torch.float64)
cos_teta = 
 tensor([[ 9.8423e-01],
        [ 9.2310e-01],
        [ 4.3197e-04],
        [ 1.2352e-01],
        [-8.9868e-01]], dtype=torch.float64)
sin_teta = 
 tensor([[-0.1769],
        [-0.3845],
        [-1.0000],
        [ 0.9923],
        [-0.4386]], dtype=torch.float64)
cos_teta = 
 tensor([[-9.8630e-01],
        [-8.3919e-01],
        [ 4.3204e-04],
        [-3.0659e-01],
        [-8.9868e-01]], dtype=torch.float64)
sin_teta = 
 tensor([[-0.1650],
        [-0.5438],
    

In [382]:
print("solutions = \n", solutions)
print(solutions.shape)  # (batch_size,4,3,4)

solutions = 
 tensor([[[[        nan,         nan,         nan,         nan],
          [        nan,         nan,         nan,         nan],
          [        nan,         nan,         nan,         nan]],

         [[        nan,         nan,         nan,         nan],
          [        nan,         nan,         nan,         nan],
          [        nan,         nan,         nan,         nan]],

         [[ 6.1485e-07,  1.0000e+00,  5.5752e-07, -4.2317e-08],
          [-1.3064e-06,  5.5752e-07, -1.0000e+00,  2.2173e-07],
          [ 6.0000e+00, -4.2317e-08, -2.2173e-07, -1.0000e+00]],

         [[-9.3345e-01, -1.0894e-01, -9.9372e-01,  2.5383e-02],
          [ 1.3489e+00,  9.8114e-01, -1.1159e-01, -1.5782e-01],
          [-4.8291e+00,  1.5966e-01,  7.7110e-03,  9.8714e-01]]],


        [[[ 1.5200e+00,  5.3597e-01, -5.1369e-01, -6.6997e-01],
          [ 1.5441e+00, -7.8592e-01, -5.9342e-01, -1.7372e-01],
          [ 1.1545e+00, -3.0834e-01,  6.1965e-01, -7.2178e-01]],

         [[ 1.

11. Reprojection of points in 2D from the newly estimated matrices to verify the estimation error.

In [383]:
def projection3D2D(point3D,C,R,A) :
  # 3D point = [ Xw, Yw, Zw ]'   (1*3)
  # T : camera translation matrix : (3*1)
  # R : camera rotation matrix : (3*3)
  # A : intraseca matrix of the camera : (3*3)
  # Output : return the coordonates of the point in 2D 

  PI = torch.cat((torch.eye(3, dtype=torch.float64),torch.zeros((3,1), dtype=torch.float64)),dim=1)  # (3*4)

  Rt = torch.cat((R,C),dim=1)               # (3*4)
  Rt = torch.cat((Rt,torch.tensor([[0,0,0,1]], dtype=torch.float64)),dim=0)   # (4*4)

  point3D_bis = torch.cat((torch.reshape(point3D,(3,1)),torch.tensor([[1]],dtype=torch.float64)),dim=0)   #(4*1)
  
  point2D = torch.tensordot(torch.tensordot(torch.tensordot(A,PI,dims=1),Rt,dims=1),point3D_bis,dims=1)  # 2D point = [u, v, w] (3*1)
  point2D = point2D / point2D[2]        # 2D point = [u, v, 1] (3*1)
  return point2D[:2]


C_transpose = C.unsqueeze(-1) # (batch_size,3,1)

p1 = vmap(projection3D2D)(P1,C_transpose,R,A)   # (batch_size,2,1)
print("p1 = ", p1)
print(p1.shape)  # (batch_size,2,1)
p2 = vmap(projection3D2D)(points3D[1],C_transpose,R,A)
print("p2 = ", p2)
p3 = vmap(projection3D2D)(points3D[2],C_transpose,R,A)
print("p3 = ", p3)
p4 = vmap(projection3D2D)(P4,C_transpose,R,A)
print("p4 = ", p4)




p1 =  tensor([[[258.2703],
         [186.3480]],

        [[276.3400],
         [ 80.1185]],

        [[631.5516],
         [117.5542]],

        [[187.1621],
         [423.8975]],

        [[218.6422],
         [534.3352]]], dtype=torch.float64)
torch.Size([5, 2, 1])
p2 =  tensor([[[362.1920],
         [101.4570]],

        [[435.9088],
         [ 63.8265]],

        [[532.9409],
         [351.3230]],

        [[384.0061],
         [206.7819]],

        [[598.2369],
         [150.9431]]], dtype=torch.float64)
p3 =  tensor([[[ 69.7793],
         [418.3200]],

        [[247.9908],
         [231.0807]],

        [[406.2896],
         [372.5063]],

        [[307.0671],
         [124.6965]],

        [[459.5029],
         [341.4061]]], dtype=torch.float64)
p4 =  tensor([[[ 177.2934],
         [-121.6081]],

        [[ 571.7495],
         [ 109.1467]],

        [[ 517.5180],
         [ 161.9754]],

        [[ 506.0176],
         [ 138.5945]],

        [[ 351.6331],
         [ 426.9156]]], d

12. Calculation of errors = distance between the 2D points estimated from the found rotation and position matrices and the 2D points from the initial matrices

In [ ]:
def distance(pt, pt_estimation):
  # input : pt = [u,v] 2D point in the image  (2,1)
  #         pt_estimation = [u_est,v_est] 2D point estimated by P3P (2,1)  

  erreur = torch.tensor(0, dtype=torch.float64)  # Initialize error as a tensor
  for i in range(len(pt)): 
    erreur = erreur + (pt[i] - pt_estimation[i])**2  
  return torch.sqrt(erreur)


def affichage_erreur(solutions,points2D,points3D,A) : 
  
  # Compute the error of estimation for each points after the P3P algorithm 

  # solutions : solution matrix returned by P3P (4*3*4)
  # points 3D : 4 pts 3D used for P3P - list of tensors (batch_size, 3)
  # points 2D : 4 pts 2D used for P3P (image of the 3D points)
  batch_size_ = solutions.shape[0]

  for k in range(batch_size_):  # Iterate over the batch size
    print("\n\n")
    print("============ Batch number : ", k+1,"=====================")

    P1 = points3D[0][k]
    P2 = points3D[1][k]
    P3 = points3D[2][k]
    P4 = points3D[3][k]
    A_ = A[k,:,:]

    erreurs = []
    nb_sol = 0

    for i in range(4):  # Iterate over the 4 solutions: 
      R = solutions[k,i,:,1:] # (3,3)
      C = solutions[k,i,:,:1] # (3,1)
      
      #if not R[0,0].isnan() : 
      nb_sol += 1 
      print("------------ Solution n° : ",nb_sol,"----------------")
      print("R = \n",R,)
      print("C = \n",C,)

      
      p1_P3P = projection3D2D(P1,C,R,A_)
      p2_P3P = projection3D2D(P2,C,R,A_)
      p3_P3P = projection3D2D(P3,C,R,A_)
      p4_P3P = projection3D2D(P4,C,R,A_)
      pt_2D_P3P = [p1_P3P,p2_P3P,p3_P3P,p4_P3P]   # (4,2)


      erreurs.append([0])
      for j in range(len(points2D)):
          erreur_pt = torch.where(R[0,0].isnan(),float('inf'),distance(points2D[j][k],pt_2D_P3P[j]))
          erreurs[i]+=erreur_pt
    
    
        
    indice_min = 0
    min = erreurs[0]
    for i in range(1,len(erreurs)) :
      if erreurs[i]<min :
        min = erreurs[i]
        indice_min = i

    R_opti = solutions[k,indice_min,:,1:] 
    C_opti = solutions[k,indice_min,:,:1]
    print("\n------------ Best solution : ----------------")
    print("Solution n° :",indice_min+1,"\n")
    print("R estimé = \n", R_opti,"\n")
    print("C estimé = \n", C_opti, "\n")

affichage_erreur(solutions, [p1, p2, p3, p4], [P1,P2,P3, P4], A)




============ Batch number :  1 =====================
------------ Solution n° :  1 ----------------
R = 
 tensor([[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]], dtype=torch.float64)
C = 
 tensor([[nan],
        [nan],
        [nan]], dtype=torch.float64)
tensor([[258.2703],
        [186.3480]], dtype=torch.float64)
tensor([[362.1920],
        [101.4570]], dtype=torch.float64)
tensor([[ 69.7793],
        [418.3200]], dtype=torch.float64)
tensor([[ 177.2934],
        [-121.6081]], dtype=torch.float64)
------------ Solution n° :  2 ----------------
R = 
 tensor([[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]], dtype=torch.float64)
C = 
 tensor([[nan],
        [nan],
        [nan]], dtype=torch.float64)
tensor([[258.2703],
        [186.3480]], dtype=torch.float64)
tensor([[362.1920],
        [101.4570]], dtype=torch.float64)
tensor([[ 69.7793],
        [418.3200]], dtype=torch.float64)
tensor([[ 177.2934],
        [-121.6081]], dtype=torch.float64